In [3]:
def parse_data(filename):
    testlog={}
    with open(filename, 'r') as f:
        for line in f:
            line=line.strip()
            if not line:
                continue
            if line.startswith("Running"):
                curtest=line.split('/')[-1].split('.')[0]
                testlog[curtest]=([], [], [])
            elif line.startswith("--Using comparator:"):
                curvah=line.split(' ')[-1].replace('-', '')
            elif line.startswith("Explored"):
                explored_nodes=int(line.split(' ')[-1])
                testlog[curtest][0].append(explored_nodes)
            elif line.startswith("Backtracks"):
                backtracks=int(line.split(' ')[-1])
                testlog[curtest][1].append(backtracks)
            elif line.startswith("Time"):
                time=float(line.split(' ')[-1])
                testlog[curtest][2].append(time)
    return testlog
data_fc = parse_data("withfc.txt")
data_bt = parse_data("withoutfc.txt")
print(data_fc)

{'d-10-01': ([24379, 93671906, 665, 1002, 120250533], [19088, 115662652, 296, 382, 120653001], [102.932572, 106583.690011, 2.999941, 4.464361, 147630.699658]), 'd-10-06': ([451284, 2214587, 84364, 83083, 7044944], [362926, 3129324, 50933, 51488, 8096524], [498.279715, 2425.255066, 87.605496, 87.006991, 8359.820252]), 'd-10-07': ([143859, 46987974, 10424, 15372, 45195402], [105117, 55151903, 6717, 11173, 45973700], [155.58115, 52094.803179, 14.834497, 17.27403, 53808.870652]), 'd-10-08': ([524365, 2219501, 2318987, 2322561, 255641], [481008, 3071439, 1815075, 1840567, 302907], [582.316691, 2475.12289, 2437.652453, 2457.455625, 300.533402]), 'd-10-09': ([30875, 2387247067, 1328349, 1711481, 569883266], [27011, 3868210293, 1004392, 1319629, 749575720], [33.05554, 2641196.001217, 1370.634878, 1795.98435, 668437.709968])}


In [4]:
for k,v in data_fc.items():
    print(k)
    print(v)

d-10-01
([24379, 93671906, 665, 1002, 120250533], [19088, 115662652, 296, 382, 120653001], [102.932572, 106583.690011, 2.999941, 4.464361, 147630.699658])
d-10-06
([451284, 2214587, 84364, 83083, 7044944], [362926, 3129324, 50933, 51488, 8096524], [498.279715, 2425.255066, 87.605496, 87.006991, 8359.820252])
d-10-07
([143859, 46987974, 10424, 15372, 45195402], [105117, 55151903, 6717, 11173, 45973700], [155.58115, 52094.803179, 14.834497, 17.27403, 53808.870652])
d-10-08
([524365, 2219501, 2318987, 2322561, 255641], [481008, 3071439, 1815075, 1840567, 302907], [582.316691, 2475.12289, 2437.652453, 2457.455625, 300.533402])
d-10-09
([30875, 2387247067, 1328349, 1711481, 569883266], [27011, 3868210293, 1004392, 1319629, 749575720], [33.05554, 2641196.001217, 1370.634878, 1795.98435, 668437.709968])


In [5]:
import pandas as pd
import numpy as np
import altair as alt

In [22]:
df_fc = pd.DataFrame(index=data_fc.keys(), columns=['VAH1', 'VAH2', 'VAH3', 'VAH4', 'VAH5'])
df_bt = pd.DataFrame(index=data_bt.keys(), columns=['VAH1', 'VAH2', 'VAH3', 'VAH4', 'VAH5'])

for k,v in data_fc.items():
    df_fc.loc[k] = v[0]
for k,v in data_bt.items():
    df_bt.loc[k] = v[0] 
def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['Test Case', 'Heuristic', 'Explored']
    df['Solver'] = name
    return df

df1 = prep_df(df_fc, 'FC')
df2 = prep_df(df_bt, 'BT')

bt_nodes=[]
time=[]

for k,v in data_fc.items():
    bt_nodes = bt_nodes + v[1]
    time = time + [round(i) for i in v[2]]
for k,v in data_bt.items():
    bt_nodes = bt_nodes + v[1]
    time = time + [round(i) for i in v[2]]
df = pd.concat([df1, df2])
df = df.assign(Backtracks = bt_nodes, Time = time)
df=df.reindex(columns=['Test Case', 'Heuristic', 'Explored', 'Backtracks', 'Time', 'Solver'])
df=df.sort_values(['Test Case', 'Solver'], ascending=[True, False])
df=df.reset_index(drop=True)
print(df)

   Test Case Heuristic    Explored  Backtracks     Time Solver
0    d-10-01      VAH1       24379       19088      103     FC
1    d-10-01      VAH2    93671906   115662652   106584     FC
2    d-10-01      VAH3         665         296        3     FC
3    d-10-01      VAH4        1002         382        4     FC
4    d-10-01      VAH5   120250533   120653001   147631     FC
5    d-10-01      VAH1      450473     1096627      299     BT
6    d-10-01      VAH2     2928097     7424685     1815     BT
7    d-10-01      VAH3      217384      588202      137     BT
8    d-10-01      VAH4      217384      588202      144     BT
9    d-10-01      VAH5    69455201   156923151    51293     BT
10   d-10-06      VAH1      451284      362926      498     FC
11   d-10-06      VAH2     2214587     3129324     2425     FC
12   d-10-06      VAH3       84364       50933       88     FC
13   d-10-06      VAH4       83083       51488       87     FC
14   d-10-06      VAH5     7044944     8096524     8360

In [25]:
print(df.to_latex(index=False, escape=True))

\begin{tabular}{lllrrl}
\toprule
Test Case & Heuristic &   Explored &  Backtracks &    Time & Solver \\
\midrule
  d-10-01 &      VAH1 &      24379 &       19088 &     103 &     FC \\
  d-10-01 &      VAH2 &   93671906 &   115662652 &  106584 &     FC \\
  d-10-01 &      VAH3 &        665 &         296 &       3 &     FC \\
  d-10-01 &      VAH4 &       1002 &         382 &       4 &     FC \\
  d-10-01 &      VAH5 &  120250533 &   120653001 &  147631 &     FC \\
  d-10-01 &      VAH1 &     450473 &     1096627 &     299 &     BT \\
  d-10-01 &      VAH2 &    2928097 &     7424685 &    1815 &     BT \\
  d-10-01 &      VAH3 &     217384 &      588202 &     137 &     BT \\
  d-10-01 &      VAH4 &     217384 &      588202 &     144 &     BT \\
  d-10-01 &      VAH5 &   69455201 &   156923151 &   51293 &     BT \\
  d-10-06 &      VAH1 &     451284 &      362926 &     498 &     FC \\
  d-10-06 &      VAH2 &    2214587 &     3129324 &    2425 &     FC \\
  d-10-06 &      VAH3 &      84364 

C:\Users\anwar\AppData\Local\Temp\ipykernel_14496\1870780046.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(index=False, escape=True))


In [ ]:
df_fc = pd.DataFrame(index=data_fc.keys(), columns=['VAH1', 'VAH2', 'VAH3', 'VAH4', 'VAH5'])
df_bt = pd.DataFrame(index=data_bt.keys(), columns=['VAH1', 'VAH2', 'VAH3', 'VAH4', 'VAH5'])

for k,v in data_fc.items():
    df_fc.loc[k] = np.log(v[0])
for k,v in data_bt.items():
    df_bt.loc[k] = np.log(v[0]) 
def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['c1', 'c2', 'values']
    df['Legend'] = name
    return df

df1 = prep_df(df_fc, 'FC')
df2 = prep_df(df_bt, 'BT')
print(df)

In [23]:
chart = alt.Chart(df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('c2:N', title=None),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(values):Q',
        axis=alt.Axis(
            grid=False,
            title=('Backtraced Nodes'))),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('c1:N', title='Test Cases'),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Legend:N',
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                # range=['#96ceb4', '#ffcc5c'],
                # range=['#ffae49', '#5bbbe9'],
                range=['#f88c24', '#227f9f'],
            ),
        ))\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0    
    )
chart.display(renderer='svg')

d:\playground\rumana_assignment\lib\site-packages\altair\utils\core.py:317: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for col_name, dtype in df.dtypes.iteritems():


alt.Chart(...)